In [1]:
import numpy as np
import random
from walksat_package import*
import time
import math
from multiprocessing import Process
import concurrent.futures
import itertools
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
import pickle

- First verify the existence of the load_dir (where error model files gets loaded from and is set to the paper_data folder by default), save_dir (where you want to save the walksat result data, make sure to set it anything other than the paper_data folder to avoid overwriting those files) and sat_dir (the directory where the uniform random 3-SAT files are to be stored)
- The error model files read by the get_errorMap function are in pickle format and therefore need to be loaded using pickle.load() function. It loads a 17x2 numpy array.
- Error model data array has 17 rows corresponding to 17 distinct values of break-value that can be encountered during walksat simulation (from 0 to 16). The first column stores the mean of the measured break-values that for the ideal value denoted by the row index. The second column stores the standard deviation of the measured break-values. So the (3,0) element would store the mean of measured break-value at ideal value = 3 and (4,1) would store the standard deviation of measured break value at ideal value = 4.
- Two kinds of error model files are made available: 
- (A) bv_vs_nvar_errormap_NX.csv that contain error model data for N = X (for fixed memristor tuning errors of 3uS and 0.25uS at ON and OFF states respectively)
- (B) bv_vs_sigma_errormap_NX_Gh_sigma_Y_Gl_sigma_Z.csv that contain error model data for N = X, std at ON state = Y and std at OFF state = Z.

In [8]:
load_dir = './data/'
save_dir = './new_data/'
sat_dir = './sat/'

In [3]:
################## Running WalkSAT/SKC simulator on 3-SAT problems: Starting Example ##################

###The instances chosen are decided by the instance_start_list and instance_end_list, if they are 1 and 80, then all instances
###with indices 1 to 80 will be selected and evaluated.

nvar = 14 #variable size
p = 0.35 #optimal p value for the specific problem size
instance_start = 1 #Index of starting instance
instance_end = 80 #Index of ending instance;
max_iter = 500
max_flips = 10000
#sim_type determines the type of error model to be used, for ideal walksat simulation just set it to zero.
sim_type = 0
num_instances = instance_end - instance_start + 1
full_data_arr = np.zeros((num_instances,max_iter))
err_map = get_errorMap(load_dir,sim_type)
params = {'max_iter' : max_iter, 'max_flip' : max_flips, 'p' : p, 'err_map' : err_map, 'sat_dir' : sat_dir}
arg_list = []
for j in np.arange(instance_start,instance_end+1,1):
    arg_list.append(j)

instance_counter = 0
instance_stat = np.zeros((num_instances,4))
with concurrent.futures.ProcessPoolExecutor() as executor:
    pool = executor.map(WalkSat_Solver_full, itertools.repeat(nvar), arg_list, itertools.repeat(params))
    for res in pool:
        instance_stat[instance_counter,0] = res[4] #instance-wise TTS
        instance_stat[instance_counter,1] = res[0] #Average flips in successful iterations
        instance_stat[instance_counter,2] = res[1] #Success Rate = Num of successful iterations/MAX_ITER
        instance_stat[instance_counter,3] = res[0]+((1-res[1])/(res[1]))*max_flips #effective run-length
        full_data_arr[instance_counter,] = res[5][:,0].T
        instance_counter = instance_counter + 1     

print("Ideal WalkSAT/SKC simulation results:")
print("N="+str(nvar)+"; Median TTS="+str(np.median(instance_stat[:,0]))+"; Success Rate="+str(np.mean(instance_stat[:,2])))


Ideal WalkSAT/SKC simulation results:
N=14; Median TTS=167.0; Success Rate=1.0


In [3]:
################## Running WalkSAT/SKC simulator on 3-SAT problems with non-ideal and ideal memristor conductances ###########
############################## Figure 5 of main text and Figure S21a of Supplementary Information################################# 

###The instances chosen are decided by the instance_start_list and instance_end_list, if they are 1 and 80, then all instances
###with indices 1 to 80 will be selected and evaluated.

###Uncomment the below lines if only one variable size is to be simulated
#---------------------------------------------------
p_opt = [0.35] #optimal p value for the specific problem size
nvar_list = [14] #variable size
instance_start_list = [1] #Index of starting instance
instance_end_list = [80] #Index of ending instance;
#---------------------------------------------------

###Uncomment the below lines if multiple variable sizes are to be simulated
#---------------------------------------------------
#p_opt = [0.35,0.35,0.4,0.4,0.45]
#nvar_list = [14,20,50,75,100]
#instance_start_list = [1,921,921,51,921]
#instance_end_list = [80,1000,1000,100,1000]
#---------------------------------------------------

max_iter = 500
max_flips = 10000

###  sim_type decides the type of simulation to be done:
###  0 -- Perfect memristor programmin, no variations
###  1 -- non-ideal memristor programming (std at Gon = 3uS, std at Goff = 0.25uS), error model is read by the get_errorMap() function.
###       Argument required: N; Usage: get_errorMap(1,N=14)
###  2 -- non-ideal memristor programming for different std values, error model is read by the get_errorMap() function.
###       Arguments required: Gh_sigma, Gl_sigma; Usage: get_errorMap(2,Gh_sigma=3e-6,Gl_sigma=0.25e-6)            
sim_type = 1
num_instance_sets = len(nvar_list)
tts_vs_nvar_real = np.zeros((num_instance_sets,6))
tts_vs_nvar_ideal = np.zeros((num_instance_sets,6))
for i in range(num_instance_sets):
    
    print("Evaluating Instance set (Real) of V: "+str(nvar_list[i]))
    num_instances = instance_end_list[i] - instance_start_list[i] + 1
    full_data_arr = np.zeros((num_instances,max_iter))
    err_map = get_errorMap(load_dir,sim_type,N=nvar_list[i])
    params = {'max_iter' : max_iter, 'max_flip' : max_flips, 'p' : p_opt[i], 'err_map' : err_map, 'sat_dir' : sat_dir}
    arg_list = []
    for j in np.arange(instance_start_list[i],instance_end_list[i]+1,1):
        arg_list.append(j)
        
    instance_counter = 0
    instance_stat = np.zeros((num_instances,4))
    with concurrent.futures.ProcessPoolExecutor() as executor:
        pool = executor.map(WalkSat_Solver_full, itertools.repeat(nvar_list[i]), arg_list, itertools.repeat(params))
        for res in pool:
            instance_stat[instance_counter,0] = res[4] #instance-wise TTS
            instance_stat[instance_counter,1] = res[0] #Average flips in successful iterations
            instance_stat[instance_counter,2] = res[1] #Success Rate = Num of successful iterations/MAX_ITER
            instance_stat[instance_counter,3] = res[0]+((1-res[1])/(res[1]))*max_flips #effective run-length
            full_data_arr[instance_counter,] = res[5][:,0].T
            instance_counter = instance_counter + 1     
            
    max_flips_opt = get_optMaxflips(max_flips,full_data_arr,0.99) #optimize the max_flips across all instances of this problem size
    instance_stat = revData_with_optMaxflips(max_flips_opt,full_data_arr,0.99) #revise data (average # of flips, success rate etc) with the new optimized max_flips
    
    tts_vs_nvar_real[i,0] = nvar_list[i]
    tts_vs_nvar_real[i,1] = np.median(instance_stat[:,0]) #median batch TTS
    tts_vs_nvar_real[i,2] = np.mean(instance_stat[:,0]) #mean batch TTS
    tts_vs_nvar_real[i,3] = np.median(instance_stat[:,3]) #median batch effective run-length
    tts_vs_nvar_real[i,4] = np.mean(instance_stat[:,3]) #mean batch effective run-length
    tts_vs_nvar_real[i,5] = max_flips_opt #optimized max_flips for the problem size
            
    print("Real; N="+str(nvar_list[i])+"; TTS="+str(tts_vs_nvar_real[i,1])+"; MR="+str(tts_vs_nvar_real[i,3]))
    
    with open(save_dir+'all_real_data_wsat_N'+str(nvar_list[i])+'.csv','wb') as f:
        pickle.dump([instance_stat,full_data_arr],f)
        
    #################################################################################################################################

    print("Evaluating Instance set (Ideal) of V: "+str(nvar_list[i]))
    sim_type = 0
    num_instances = instance_end_list[i] - instance_start_list[i] + 1
    full_data_arr = np.zeros((num_instances,max_iter))
    err_map = get_errorMap(load_dir,sim_type)
    params = {'max_iter' : max_iter, 'max_flip' : max_flips, 'p' : p_opt[i], 'err_map' : err_map, 'sat_dir' : sat_dir}
    arg_list = []
    for j in np.arange(instance_start_list[i],instance_end_list[i]+1,1):
        arg_list.append(j)
        
    instance_counter = 0
    instance_stat = np.zeros((num_instances,4))
    with concurrent.futures.ProcessPoolExecutor() as executor:
        pool = executor.map(WalkSat_Solver_full, itertools.repeat(nvar_list[i]), arg_list, itertools.repeat(params))
        for res in pool:
            instance_stat[instance_counter,0] = res[4]
            instance_stat[instance_counter,1] = res[0]
            instance_stat[instance_counter,2] = res[1]
            instance_stat[instance_counter,3] = res[0]+((1-res[1])/(res[1]))*max_flips
            full_data_arr[instance_counter,] = res[5][:,0].T
            instance_counter = instance_counter + 1     
    
    max_flips_opt = get_optMaxflips(max_flips,full_data_arr,0.99) #optimize the max_flips across all instances of this problem size
    instance_stat = revData_with_optMaxflips(max_flips_opt,full_data_arr,0.99) #revise data (average # of flips, success rate etc) with the new optimized max_flips
    
    tts_vs_nvar_ideal[i,0] = nvar_list[i]
    tts_vs_nvar_ideal[i,1] = np.median(instance_stat[:,0])
    tts_vs_nvar_ideal[i,2] = np.mean(instance_stat[:,0])
    tts_vs_nvar_ideal[i,3] = np.median(instance_stat[:,3])
    tts_vs_nvar_ideal[i,4] = np.mean(instance_stat[:,3])
    tts_vs_nvar_ideal[i,5] = max_flips_opt #optimized max_flips for the problem size
            
    print("Ideal; N="+str(nvar_list[i])+"; TTS="+str(tts_vs_nvar_ideal[i,1])+"; MR="+str(tts_vs_nvar_ideal[i,3]))
    
    with open(save_dir+'all_ideal_data_wsat_N'+str(nvar_list[i])+'.csv','wb') as f:
        pickle.dump([instance_stat,full_data_arr],f)
    
with open(save_dir+'tts_vs_nvar_real_wsat.csv','wb') as f:
    pickle.dump([tts_vs_nvar_real],f)
with open(save_dir+'tts_vs_nvar_ideal_wsat.csv','wb') as f:
    pickle.dump([tts_vs_nvar_ideal],f)

Evaluating Instance set (Real) of V: 14
Real; N=14; TTS=160.4640594083376; MR=36.744481734865104
Evaluating Instance set (Ideal) of V: 14
Ideal; N=14; TTS=152.20540334037318; MR=36.834295195498484


In [7]:
################## Running WalkSAT/SKC simulator on 3-SAT problems with non-ideal and ideal memristor conductances ###########
############################## Figure S21b of Supplementary Information################################# 

###Uncomment the below lines if only one variable size is to be simulated
#---------------------------------------------------
Gh_sigma_list = [3e-6] #List of ON state memristor tuning standard deviation values
Gl_sigma_list = [0.25e-6] #List of OFF state memristor tuning standard deviation values
#---------------------------------------------------

###Uncomment the below lines if multiple variable sizes are to be simulated
#---------------------------------------------------
#Gh_sigma_list = [0,3e-6,3e-6,3e-6,10e-6,20e-6,30e-6,30e-6]
#Gl_sigma_list = [0,0.25e-6,1e-6,3e-6,3e-6,3e-6,3e-6,5e-6]
#---------------------------------------------------

p_opt = 0.35
nvar = 20
instance_start = 921
instance_end = 1000
num_instances = instance_end - instance_start + 1
max_iter = 500
max_flips = 50000
sim_type = 2
num_instance_sets = len(Gh_sigma_list)
tts_vs_sigma = np.zeros((num_instance_sets,7))

for i in range(num_instance_sets):
    
    print("Evaluating Instance set (Real) of Gh_sigma: "+str(Gh_sigma_list[i])+"; Gl_sigma: "+str(Gl_sigma_list[i]))
    full_data_arr = np.zeros((num_instances,max_iter))
    err_map = get_errorMap(load_dir,sim_type,N=nvar,Gh_sigma=Gh_sigma_list[i],Gl_sigma=Gl_sigma_list[i])
    params = {'max_iter' : max_iter, 'max_flip' : max_flips, 'p' : p_opt, 'err_map' : err_map, 'sat_dir' : sat_dir}
    arg_list = []
    for j in np.arange(instance_start,instance_end+1,1):
        arg_list.append(j)
        
    instance_counter = 0
    instance_stat = np.zeros((num_instances,4))
    with concurrent.futures.ProcessPoolExecutor() as executor:
        pool = executor.map(WalkSat_Solver_full, itertools.repeat(nvar), arg_list, itertools.repeat(params))
        for res in pool:
            instance_stat[instance_counter,0] = res[4]
            instance_stat[instance_counter,1] = res[0]
            instance_stat[instance_counter,2] = res[1]
            instance_stat[instance_counter,3] = res[0]+((1-res[1])/(res[1]))*max_flips
            full_data_arr[instance_counter,] = res[5][:,0].T
            instance_counter = instance_counter + 1   
            
    max_flips_opt = get_optMaxflips(max_flips,full_data_arr,0.99) #optimize the max_flips across all instances of this problem size
    instance_stat = revData_with_optMaxflips(max_flips_opt,full_data_arr,0.99) #revise data (average # of flips, success rate etc) with the new optimized max_flips
            
    tts_vs_sigma[i,0] = Gh_sigma_list[i]
    tts_vs_sigma[i,1] = Gl_sigma_list[i]
    tts_vs_sigma[i,2] = np.median(instance_stat[:,0])
    tts_vs_sigma[i,3] = np.mean(instance_stat[:,0])
    tts_vs_sigma[i,4] = np.median(instance_stat[:,3])
    tts_vs_sigma[i,5] = np.mean(instance_stat[:,3])
    tts_vs_sigma[i,6] = max_flips_opt
            
    print("Real; Gh_sigma="+str(Gh_sigma_list[i])+"; Gl_sigma="+str(Gl_sigma_list[i])+"; TTS="+str(tts_vs_sigma[i,3])+"; MR="+str(tts_vs_sigma[i,5]))
    
    with open(save_dir+'all_real_data_wsat_N20_'+str(Gh_sigma_list[i])+'_'+str(Gl_sigma_list[i])+'.csv','wb') as f:
        pickle.dump([instance_stat,full_data_arr],f)
    
with open(save_dir+'tts_vs_sigma_wsat.csv','wb') as f:
    pickle.dump([tts_vs_sigma],f)

Evaluating Instance set (Real) of Gh_sigma: 3e-06; Gl_sigma: 2.5e-07
Real; Gh_sigma=3e-06; Gl_sigma=2.5e-07; TTS=368.94826700063567; MR=82.81697149715092
